![Görüntü Açıklaması](bafgan.png)

 ## Gerekli Kütüphanelerin Eklenmesi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
import pickle

 ## Veri Seti İçin Ön İşlemler

In [2]:
df = pd.read_csv('base.csv')  
data = pd.read_csv('base.csv') 

categorical_columns = ['income', 'customer_age', 'payment_type', 'employment_status', 'housing_status', 'source', 'device_os', 'device_distinct_emails_8w', 'device_fraud_count', 'month']

encoder = OneHotEncoder(sparse=False, drop='first')
encoded_df = pd.DataFrame(encoder.fit_transform(df[categorical_columns]))

encoded_df.columns = encoder.get_feature_names_out(categorical_columns)

df = df.drop(columns=categorical_columns)

df = pd.concat([df, encoded_df], axis=1)

fraud_bool = df['fraud_bool']

binary_columns = ["email_is_free","phone_home_valid","phone_mobile_valid","has_other_cards","foreign_request","keep_alive_session"]  # Binary sütunların isimleri
columns_to_scale = df.columns.difference(['fraud_bool'] + binary_columns)
scaler = MinMaxScaler()
scaled_df = df.copy()
scaled_df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

scaled_df['fraud_bool'] = fraud_bool
scaled_df[binary_columns] = df[binary_columns] 

 ## Modelin Kurulması ve Kaydedilmesi

In [ ]:
latent_dim = 1024
num_epochs = 100
batch_size = 64

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

generator = Generator(input_dim=latent_dim, output_dim=scaled_df.shape[1]).to(device)
discriminator = Discriminator(input_dim=scaled_df.shape[1]).to(device)

optimizer_g = optim.Adam(generator.parameters(), lr=0.0003, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.00005, betas=(0.5, 0.999))
criterion = nn.BCELoss()

data_tensor = torch.tensor(scaled_df.values, dtype=torch.float32).to(device)
train_loader = DataLoader(TensorDataset(data_tensor), batch_size=batch_size, shuffle=True)

def train_model():
    start_time = time.time()
    for epoch in range(num_epochs):
        for real_data in train_loader:
            real_data = real_data[0]

            batch_size = real_data.size(0)
            real_labels = torch.full((batch_size, 1), 0.9, device=device) 
            fake_labels = torch.full((batch_size, 1), 0.1, device=device) 

            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = generator(z)

            for _ in range(2):  
                optimizer_d.zero_grad()
                outputs = discriminator(real_data)
                d_loss_real = criterion(outputs, real_labels)
                d_loss_real.backward()

                outputs = discriminator(fake_data.detach())
                d_loss_fake = criterion(outputs, fake_labels)
                d_loss_fake.backward()
                optimizer_d.step()

            # Generator eğitimi
            optimizer_g.zero_grad()
            outputs = discriminator(fake_data)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()  
            optimizer_g.step()

        print(f'Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss_real.item() + d_loss_fake.item()} | G Loss: {g_loss.item()}')

    end_time = time.time()
    return end_time - start_time

if device.type == 'cuda':
    gpu_time = train_model()
    print(f"GPU eğitim süresi: {gpu_time:.2f} saniye")
else:
    cpu_time = train_model()
    print(f"CPU eğitim süresi: {cpu_time:.2f} saniye")

with open('generator_model.pkl', 'wb') as f:
    pickle.dump(generator.state_dict(), f)

with open('discriminator_model.pkl', 'wb') as f:
    pickle.dump(discriminator.state_dict(), f)


Epoch [1/100] | D Loss: 0.6900216937065125 | G Loss: 2.2028658390045166
Epoch [2/100] | D Loss: 0.7642569541931152 | G Loss: 1.5859657526016235
Epoch [3/100] | D Loss: 0.735119640827179 | G Loss: 2.0101470947265625
Epoch [4/100] | D Loss: 0.6549120545387268 | G Loss: 2.095292568206787
Epoch [5/100] | D Loss: 0.6535765826702118 | G Loss: 2.0708212852478027
Epoch [6/100] | D Loss: 0.6511710286140442 | G Loss: 2.0806498527526855
Epoch [7/100] | D Loss: 0.6508444547653198 | G Loss: 2.090548515319824
Epoch [8/100] | D Loss: 0.6835614740848541 | G Loss: 2.0646584033966064
Epoch [9/100] | D Loss: 0.6502811908721924 | G Loss: 2.0837135314941406
Epoch [10/100] | D Loss: 0.6503421068191528 | G Loss: 2.095125913619995
Epoch [11/100] | D Loss: 0.6502617597579956 | G Loss: 2.0862958431243896
Epoch [12/100] | D Loss: 0.6506488025188446 | G Loss: 2.0837883949279785
Epoch [13/100] | D Loss: 0.6502555906772614 | G Loss: 2.088930606842041
Epoch [14/100] | D Loss: 0.6502152979373932 | G Loss: 2.08029818